In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Generate gdf on leg level

In [2]:
# create geojson of linestrings
root = "W:/Master-Thesis-Repository/"
df = pd.DataFrame()
files = ["freemove_etappen_koordinaten_602.csv", "freemove_etappen_koordinaten_603.csv", "freemove_etappen_koordinaten_604.csv", "freemove_etappen_koordinaten_605.csv"]
for file in files:
    path = root + "data/freemove_dlr_data/raw/" + file
    df = pd.concat([df, pd.read_csv(path, delimiter=";")])

df = df.groupby(['CAMPAIGN_ID', 'PERSON_ID', 'TRIP_ID']).apply(lambda g: g.sort_values('LEG_START')).reset_index(drop=True)

# Generate linestrings per Leg
df["wkt"] = "LINESTRING(" + df["LEG_SHAPE"].str.strip(("[")).str.strip(("]")) + ")"

geometry_leg = gpd.GeoSeries.from_wkt(df['wkt'])

gdf = gpd.GeoDataFrame(df, geometry=geometry_leg)[["PERSON_ID", "TRIP_ID", "LEG_ID", "LEG_START", "LEG_END", "TRIP_PURPOSE_IDS", "TRAFFIC_MODE", "LEG_LEN_IN_MTRS", "LEG_DURATION_IN_SECS", "geometry"]]

# Generate start and endpoints
gdf['LEG_SP'], gdf['LEG_EP'] = gdf['geometry'].apply(lambda x: str(x.coords[0])), gdf['geometry'].apply(lambda x: str(x.coords[-1]))

gdf

,PERSON_ID,TRIP_ID,LEG_ID,LEG_START,LEG_END,TRIP_PURPOSE_IDS,TRAFFIC_MODE,LEG_LEN_IN_MTRS,LEG_DURATION_IN_SECS,geometry,LEG_SP,LEG_EP
0,16348,985222,5751809,2022-10-31 09:15:57,2022-10-31 09:27:58,2,bicycle,2468,721,"LINESTRING (13.50497 52.45445, 13.50498 52.454...","(13.504967, 52.454451)","(13.526898, 52.457209)"
1,16348,985222,5751810,2022-10-31 09:27:59,2022-10-31 09:34:08,2,walk,106,369,"LINESTRING (13.52690 52.45719, 13.52690 52.457...","(13.526902, 52.457195)","(13.527252, 52.457538)"
2,16348,985223,5751811,2022-10-31 11:04:22,2022-10-31 11:09:12,7,walk,353,290,"LINESTRING (13.52685 52.45699, 13.52685 52.457...","(13.526849, 52.45699)","(13.525136, 52.455497)"
3,16348,985301,5752119,2022-10-31 11:38:24,2022-10-31 11:43:51,7,walk,315,327,"LINESTRING (13.52438 52.45529, 13.52433 52.455...","(13.524382, 52.455285)","(13.526203, 52.456038)"
4,16348,985302,5752120,2022-10-31 12:01:50,2022-10-31 12:06:40,2,walk,318,290,"LINESTRING (13.52601 52.45596, 13.52601 52.455...","(13.526006, 52.455956)","(13.526667, 52.457062)"
...,...,...,...,...,...,...,...,...,...,...,...,...
4953,17273,1012443,5860208,2022-11-15 09:43:13,2022-11-15 09:49:54,1,public_transport_train,3120,401,"LINESTRING (13.40486 52.49841, 13.40482 52.498...","(13.40486, 52.498411)","(13.360306, 52.500437)"
4954,17273,1012443,5860211,2022-11-15 09:49:55,2022-11-15 09:55:12,1,walk,499,317,"LINESTRING (13.36033 52.50048, 13.36034 52.500...","(13.360332, 52.50048)","(13.355352, 52.502036)"
4955,17273,1012443,5860236,2022-11-15 09:55:12,2022-11-15 09:55:17,NaN,walk,7,5,"LINESTRING (13.35535 52.50204, 13.35537 52.502...","(13.355352, 52.502036)","(13.355435, 52.502068)"
4956,17273,1012443,5860237,2022-11-15 09:55:18,2022-11-15 10:00:40,NaN,stationary,8,322,"LINESTRING (13.35545 52.50207, 13.35552 52.502...","(13.355449, 52.502072)","(13.355557, 52.502048)"


In [3]:
# store geojson
gdf.to_file("W:/Master-Thesis-Repository/data/freemove_dlr_data/raw_full.geojson")

c:\Users\Bened\Documents\Git\Master-Thesis\env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# Generate gdf on trip level (concatenate legs)

In [3]:
# create geojson of linestrings
root = "W:/Master-Thesis-Repository/"
df_trip = pd.DataFrame()
files = ["freemove_etappen_koordinaten_602.csv", "freemove_etappen_koordinaten_603.csv", "freemove_etappen_koordinaten_604.csv", "freemove_etappen_koordinaten_605.csv"]
for file in files:
    path = root + "data/freemove_dlr_data/raw/" + file
    df_trip = pd.concat([df_trip, pd.read_csv(path, delimiter=";")])

df_trip = df_trip.groupby(['CAMPAIGN_ID', 'PERSON_ID', 'TRIP_ID']).apply(lambda g: g.sort_values('LEG_START')).reset_index(drop=True)

df_trip = df_trip.groupby(['CAMPAIGN_ID', 'PERSON_ID', 'TRIP_ID']).agg(list).reset_index().drop("CAMPAIGN_ID_1", axis=1)

df_trip["wkt_trip"] = df_trip['LEG_SHAPE'].apply(lambda x: ", ".join(x))
df_trip["wkt_trip"] = "LINESTRING(" + df_trip["wkt_trip"].str.replace("[", "").str.replace("]", "") + ")"

df_trip['TRIP_START'] = df_trip['LEG_START'].apply(lambda x: min(x))
df_trip['TRIP_END'] = df_trip['LEG_END'].apply(lambda x: max(x))

df_trip['TRIP_LEN_IN_MTRS'] = df_trip['LEG_LEN_IN_MTRS'].apply(lambda x: sum(x))
df_trip['TRIP_DURATION_IN_SECS'] = df_trip['LEG_DURATION_IN_SECS'].apply(lambda x: sum(x))

geometry_trip = gpd.GeoSeries.from_wkt(df_trip['wkt_trip'])

gdf_trip = gpd.GeoDataFrame(df_trip, geometry=geometry_trip)[["PERSON_ID", "TRIP_ID", "TRIP_START", "TRIP_END", "TRIP_LEN_IN_MTRS", "TRIP_DURATION_IN_SECS", "geometry"]]

# Generate start and endpoints
gdf_trip['TRIP_SP'], gdf_trip['TRIP_EP'] = gdf_trip['geometry'].apply(lambda x: str(x.coords[0])), gdf_trip['geometry'].apply(lambda x: str(x.coords[-1]))

gdf_trip

C:\Users\Bened\AppData\Local\Temp\ipykernel_21820\1544476592.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_trip["wkt_trip"] = "LINESTRING(" + df_trip["wkt_trip"].str.replace("[", "").str.replace("]", "") + ")"


,PERSON_ID,TRIP_ID,TRIP_START,TRIP_END,TRIP_LEN_IN_MTRS,TRIP_DURATION_IN_SECS,geometry,TRIP_SP,TRIP_EP
0,16348,985222,2022-10-31 09:15:57,2022-10-31 09:34:08,2574,1090,"LINESTRING (13.50497 52.45445, 13.50498 52.454...","(13.504967, 52.454451)","(13.527252, 52.457538)"
1,16348,985223,2022-10-31 11:04:22,2022-10-31 11:09:12,353,290,"LINESTRING (13.52685 52.45699, 13.52685 52.457...","(13.526849, 52.45699)","(13.525136, 52.455497)"
2,16348,985301,2022-10-31 11:38:24,2022-10-31 11:43:51,315,327,"LINESTRING (13.52438 52.45529, 13.52433 52.455...","(13.524382, 52.455285)","(13.526203, 52.456038)"
3,16348,985302,2022-10-31 12:01:50,2022-10-31 12:12:00,335,608,"LINESTRING (13.52601 52.45596, 13.52601 52.455...","(13.526006, 52.455956)","(13.526806, 52.457104)"
4,16348,985459,2022-10-31 13:23:11,2022-10-31 13:53:17,3503,1803,"LINESTRING (13.52692 52.45710, 13.52694 52.457...","(13.52692, 52.457104)","(13.500579, 52.4561)"
...,...,...,...,...,...,...,...,...,...
1403,17273,1008119,2022-11-11 09:32:50,2022-11-11 09:58:38,2335,1548,"LINESTRING (13.39846 52.51452, 13.39846 52.514...","(13.398465, 52.514515)","(13.406914, 52.498777)"
1404,17273,1008251,2022-11-11 11:23:24,2022-11-11 11:53:16,5313,1789,"LINESTRING (13.40691 52.49878, 13.40752 52.498...","(13.406914, 52.498777)","(13.354981, 52.502044)"
1405,17273,1008366,2022-11-11 15:13:35,2022-11-11 15:34:21,1806,1243,"LINESTRING (13.40748 52.48918, 13.40743 52.489...","(13.407482, 52.48918)","(13.404682, 52.495296)"
1406,17273,1011765,2022-11-14 15:54:12,2022-11-14 16:13:41,3940,1166,"LINESTRING (13.40554 52.49880, 13.40567 52.498...","(13.405542, 52.498799)","(13.355288, 52.502035)"


In [5]:
# store geojson
gdf_trip.to_file("W:/Master-Thesis-Repository/data/freemove_dlr_data/raw_full_trip.geojson")

c:\Users\Bened\Documents\Git\Master-Thesis\env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# Others

In [63]:
df["speed"] = (df.LEG_LEN_IN_MTRS / 1000) / (df.LEG_DURATION_IN_SECS/3600)

In [198]:
point_gdf = pd.DataFrame(columns=['uid', 'tid', 'leg_id', 'datetime', "lng", "lat", 'geometry'])

for i in range(0, len(gdf)):
    coords_list = [pt for pt in list(gdf.iloc[i,]['geometry'].coords)]
    point_gdf = pd.concat([ point_gdf, pd.DataFrame(data=[{"uid": gdf.iloc[i, ]["PERSON_ID"],
                                        "tid": gdf.iloc[i, ]["TRIP_ID"],
                                        "leg_id": gdf.iloc[i, ]["LEG_ID"],
                                        "datetime": gdf.iloc[i, ]["LEG_START"],
                                        "lng": coords_list[0][0],
                                        "lat": coords_list[0][1],
                                        "geometry": Point(coords_list[0])
        }])])

    point_gdf = pd.concat([ point_gdf, pd.DataFrame(data=[{"uid": gdf.iloc[i, ]["PERSON_ID"],
                                        "tid": gdf.iloc[i, ]["TRIP_ID"],
                                        "leg_id": gdf.iloc[i, ]["LEG_ID"],
                                        "datetime": gdf.iloc[i, ]["LEG_END"],
                                        "lng": coords_list[-1][0],
                                        "lat": coords_list[-1][1],
                                        "geometry": Point(coords_list[-1])
        }])])


In [199]:
point_gdf.drop("geometry", axis=1).to_csv("/Volumes/NO NAME/od_points.csv")

In [200]:
od_points = gpd.GeoDataFrame(point_gdf)

In [201]:
# store points as geojson
od_points.to_file("/Volumes/NO NAME/od_points.geojson")